### **Preentrenamiento de modelos BERT**


En este cuaderno práctico, aprenderá a construir un modelo BERT desde cero usando PyTorch.

### **Configuración**


#### **Instalación de librerías requeridas**

Las siguientes librerías requeridas **no** están preinstaladas. **Deberás ejecutar la siguiente celda** para instalarlas:


In [ ]:
#!pip install 'pandas==2.2.1'
#!pip install 'portalocker>=2.0.0'
#!pip install 'torchtext==0.16.0'
#!pip install 'pandas==2.2.1'
#!pip install transformers
#!pip install matplotlib

#### **Importación de librerías requeridas**

*Se recomienda importar todas las librerías requeridas en un solo lugar (aquí):*

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import Tensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import Transformer
from transformers import BertTokenizer
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torchtext.vocab import Vocab,build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import IMDB
import random
from itertools import chain
import pandas as pd
from copy import deepcopy
import csv
import json
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import get_linear_schedule_with_warmup

# También puede usar esta sección para suprimir advertencias generadas por su código:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')


### **Antecedentes**

#### **Introducción al preentrenamiento**


El preentrenamiento implica entrenar un modelo en un gran corpus de texto no etiquetado para capturar patrones generales del lenguaje y relaciones semánticas. Los modelos preentrenados pueden luego ajustarse finamente en tareas específicas de NLP, como análisis de sentimientos, preguntas y respuestas o traducción automática.

La motivación detrás del preentrenamiento de transformers es abordar las limitaciones de los enfoques tradicionales que requieren cantidades significativas de datos etiquetados para cada tarea específica. El preentrenamiento aprovecha la abundancia de datos de texto no etiquetados disponibles en Internet y facilita el aprendizaje por transferencia, donde el conocimiento aprendido en una tarea puede transferirse para ayudar a resolver otras tareas relacionadas.

Los objetivos de preentrenamiento juegan un papel crucial en el entrenamiento de transformers. Por ejemplo, el modelado de lenguaje enmascarado (MLM) implica enmascarar aleatoriamente algunas palabras en una oración y entrenar al modelo para predecir las palabras enmascaradas en función del contexto circundante. 

Este objetivo ayuda al modelo a aprender comprensión contextual y a completar la información faltante. Otro objetivo llamado predicción de la siguiente oración (NSP) implica predecir si dos oraciones son consecutivas o seleccionadas al azar del corpus, lo que permite al modelo aprender relaciones a nivel de oración.



### **Objetivos de preentrenamiento**

Los objetivos de preentrenamiento son componentes cruciales del proceso de preentrenamiento para transformers. Estos objetivos definen las tareas en las que se entrena el modelo durante la fase de preentrenamiento, lo que le permite aprender representaciones contextuales significativas del lenguaje. 

Dos objetivos de preentrenamiento comúnmente usados son el modelado de lenguaje enmascarado (MLM) y la predicción de la siguiente oración (NSP).

1. **Modelado de lenguaje enmascarado (MLM):**
   El modelado de lenguaje enmascarado implica enmascarar aleatoriamente algunas palabras en una oración y entrenar al modelo para predecir las palabras enmascaradas en función del contexto proporcionado por las palabras circundantes (es decir, palabras que aparecen antes o después de la palabra enmascarada).

   El objetivo es permitir que el modelo aprenda comprensión contextual y complete la información faltante.
   Así es como funciona el MLM:

   * Dada una oración de entrada, se selecciona aleatoriamente un porcentaje de las palabras y se reemplazan con un token especial `[MASK]`.
   * La tarea del modelo es predecir las palabras originales que fueron enmascaradas, dadas las palabras circundantes.
   * Durante el entrenamiento, el modelo aprende a comprender la relación entre las palabras enmascaradas y el resto de la oración, capturando efectivamente la información contextual.

3. **Predicción de la siguiente oración (NSP):**
   La predicción de la siguiente oración implica entrenar al modelo para predecir si dos oraciones son consecutivas en el texto original o si se seleccionaron al azar del corpus. Este objetivo ayuda al modelo a aprender relaciones a nivel de oración y a entender la coherencia entre oraciones.
   Así es como funciona el NSP:

   * Dado un par de oraciones, el modelo se entrena para predecir si la segunda oración sigue a la primera en el texto original o si fue seleccionada al azar del corpus.
   * El modelo aprende a capturar las relaciones entre oraciones y a entender el flujo de información en el texto.

   El NSP es particularmente útil para tareas que implican comprender la relación entre múltiples oraciones, como preguntas y respuestas o clasificación de documentos. Al entrenar al modelo para predecir la coherencia de pares de oraciones, aprende a capturar las conexiones semánticas entre ellas.

*Nota: diferentes modelos preentrenados pueden utilizar variaciones o combinaciones de estos objetivos, dependiendo de la arquitectura y la configuración de entrenamiento.*

### **Preentrenamiento de un modelo BERT**

El preentrenamiento de un modelo BERT (Bidirectional Encoder Representations from Transformers) es un proceso complejo y que consume mucho tiempo, que requiere un gran corpus de datos de texto no etiquetados y recursos computacionales significativos. Sin embargo, se presenta a continuación un ejercicio simplificado para demostrar los pasos involucrados en el preentrenamiento de un modelo BERT utilizando los objetivos de modelado de lenguaje enmascarado (MLM) y predicción de la siguiente oración (NSP).

Se indicará:

* Crear cargadores de datos de entrenamiento y prueba a partir del conjunto de datos
* Preentrenar BERT usando una tarea de MLM
* Preentrenar BERT usando una tarea de NSP
* Evaluar el modelo entrenado


#### **Cargando datos**

Vamos a cargar los archivos CSV creados en el cuaderno de preparación de datos.


In [ ]:
import shutil
import os
import urllib.request
import zipfile

# Elimina instancia anterior
if os.path.isdir("bert_dataset"):
    shutil.rmtree("bert_dataset")

# Descarga
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/bZaoQD52DcMpE7-kxwAG8A.zip"
zip_path = "BERT_dataset.zip"
urllib.request.urlretrieve(url, zip_path)

# Descomprime
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall("bert_dataset")


Ahora, puedes crear un Dataset de torch usando el archivo CSV que acaba de crear:


In [ ]:
class BERTCSVDataset(Dataset):
    def __init__(self, filename):
        self.data = pd.read_csv(filename)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        try:
            
            bert_input = torch.tensor(json.loads(row['BERT Input']), dtype=torch.long)
            bert_label = torch.tensor(json.loads(row['BERT Label']), dtype=torch.long)
            segment_label = torch.tensor([int(x) for x in row['Segment Label'].split(',')], dtype=torch.long)
            is_next = torch.tensor(row['Is Next'], dtype=torch.long)
            original_text = row['Original Text']  # Si desea usarlo
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON en la fila {idx}: {e}")
            print("BERT Input:", row['BERT Input'])
            print("BERT Label:", row['BERT Label'])
            # Maneja el error, p. ej., omitiendo esta fila o usando valores predeterminados
            return None  # o algunos valores predeterminados
        
        return bert_input, bert_label, segment_label, is_next  # Incluye original_text si es necesario

A continuación, crea una función `collate` que aplique transformaciones a lotes del iterador de datos:

In [ ]:
PAD_IDX = 0
def collate_batch(batch):
    bert_inputs_batch, bert_labels_batch, segment_labels_batch, is_nexts_batch = [], [], [], []

    for bert_input, bert_label, segment_label, is_next in batch:
        # Convierte cada secuencia en un tensor y agregarla a la lista respectiva
        bert_inputs_batch.append(torch.tensor(bert_input, dtype=torch.long))
        bert_labels_batch.append(torch.tensor(bert_label, dtype=torch.long))
        segment_labels_batch.append(torch.tensor(segment_label, dtype=torch.long))
        is_nexts_batch.append(is_next)

    # Rellena las secuencias en el lote
    bert_inputs_final = pad_sequence(bert_inputs_batch, padding_value=PAD_IDX, batch_first=False)
    bert_labels_final = pad_sequence(bert_labels_batch, padding_value=PAD_IDX, batch_first=False)
    segment_labels_final = pad_sequence(segment_labels_batch, padding_value=PAD_IDX, batch_first=False)
    is_nexts_batch = torch.tensor(is_nexts_batch, dtype=torch.long)

    return bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_batch

Usando un tamaño de lote arbitrario, puede crear dataloaders de entrenamiento y prueba:


In [ ]:
import os
for root, dirs, files in os.walk("bert_dataset"):
    print(root, files)

In [ ]:
BATCH_SIZE = 2

train_dataset_path = './bert_dataset/bert_dataset/bert_train_data.csv'
test_dataset_path = './bert_dataset/bert_dataset/bert_test_data.csv'

train_dataset = BERTCSVDataset(train_dataset_path)
test_dataset = BERTCSVDataset(test_dataset_path)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

### **Creación del modelo**

En BERT, el embedding posicional, el embedding de tokens y el embedding de segmentos son tres tipos de embeddings usadas para representar los tokens de entrada en el modelo.

1. **Embeddings de tokens (token embedding):** el embedding de tokens es la representación inicial de cada token en un modelo BERT. Mapea cada token a un vector denso de tamaño fijo, típicamente llamado tamaño de embeddings. La capa de embeddings de tokens en BERT aprende las representaciones contextuales de los tokens de entrada. Estos embeddings capturan el significado semántico de los tokens y sus relaciones con otros tokens en el contexto.

2. **Embedding posicional (positional embedding):** BERT es un modelo basado en transformers que procesa los tokens de entrada en paralelo. Sin embargo, dado que los transformers no capturan inherentemente el orden de los tokens, se usa el embedding posicional para inyectar información de posición en el modelo. Añade un vector de representación a cada token que codifica su posición en la secuencia de entrada. El embedding posicional permite a BERT entender el orden secuencial de los tokens y capturar sus posiciones relativas.

3. **Embedding de segmentos (segment embedding):** BERT puede manejar pares de oraciones o secuencias con segmentos distintos. Para diferenciar entre diferentes segmentos, como oraciones o secciones de un documento, se usa el embedding de segmentos. Asigna una representación vectorial única a cada segmento o parte de la entrada. Los embeddings de segmentos ayudan a BERT a entender las relaciones entre diferentes segmentos y a capturar el contexto dentro y entre ellos.

In [ ]:
EMBEDDING_DIM = 10

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Define la clase PositionalEncoding como un módulo de PyTorch para agregar información posicional a los embeddings  de tokens
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        # Crea una matriz de codificación posicional según la fórmula del artículo del Transformer
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        #  Aplica las codificaciones posicionales a los embeddings de tokens de entrada

        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [ ]:
class BERTEmbedding (nn.Module):

    def __init__(self, vocab_size, emb_size ,dropout=0.1,train=True):

        super().__init__()

        self.token_embedding = TokenEmbedding( vocab_size,emb_size )
        self.positional_encoding = PositionalEncoding(emb_size,dropout)
        self.segment_embedding = nn.Embedding(3, emb_size)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, bert_inputs, segment_labels=False):
        my_embeddings=self.token_embedding(bert_inputs)
        if self.train:
          x = self.dropout(my_embeddings + self.positional_encoding(my_embeddings) + self.segment_embedding(segment_labels))
        else:
          x = my_embeddings + self.positional_encoding(my_embeddings)

        return x

Ahora, define un modelo BERT completo con los siguientes componentes clave:

1. **Inicialización:** La clase `BERT` se define como una subclase de `torch.nn.Module`. Inicializa el modelo BERT con parámetros como tamaño de vocabulario, dimensión del modelo, número de capas, número de cabezas de atención y tasa de abandono (dropout).

2. **Capa de embeddings:** El modelo BERT incluye una capa de embedding que combina embeddings de tokens e embeddings de segmentos usando la clase `BERTEmbedding`.

3. **Codificador Transformer:** Se usan capas de codificador Transformer para codificar los embeddings de entrada. El número de capas, cabeceras de atención, tasa de dropout y dimensión del modelo se especifican según los parámetros definidos.

4. **Predicción de la siguiente oración (next sentence prediction):** El modelo tiene una capa lineal para *predicción de la siguiente oración*. Toma la salida del codificador Transformer y predice la relación entre dos oraciones consecutivas, clasificándolas en dos clases.

5. **Modelado de lenguaje enmascarado (masked language modeling):** El modelo también incluye una capa lineal para modelado de lenguaje enmascarado. Predice los tokens enmascarados en la secuencia de entrada tomando la salida del codificador Transformer y realizando predicciones a lo largo del vocabulario.

6. **Paso hacia adelante (forward pass):** El método `forward` define el paso hacia adelante del modelo BERT. Toma tokens de entrada (`bert_inputs`) y etiquetas de segmentos (`segment_labels`) y devuelve predicciones para las tareas de *predicción de la siguiente oración* y *modelado de lenguaje enmascarado*.



Usando un ejemplo de entrada, analicemos el embeddings en sus tres componentes esenciales: embeddings de token, codificación posicional e embeddings de segmento para comprender el proceso:


In [ ]:
VOCAB_SIZE=147161
batch = 2
count = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# carga lotes de muestra del dataloader
for batch in train_dataloader:
    bert_inputs, bert_labels, segment_labels, is_nexts = [b.to(device) for b in batch]
    count += 1
    if count == 5:
        break

In [ ]:
bert_inputs.shape

In [ ]:
#selecciona una entrada de muestra
bert_inputs[:,0]

In [ ]:
segment_labels.shape

In [ ]:
segment_labels[:,0]

In [ ]:
# Instancia el TokenEmbedding 
token_embedding = TokenEmbedding(VOCAB_SIZE, emb_size=EMBEDDING_DIM )

#los embeddings de tokens para una entrada de muestra
t_embeddings = token_embedding(bert_inputs)
# Cada token se transforma en un tensor de tamaño emb_size
print(f"Dimension de embeddings de token: {t_embeddings.size()}") # Esperado: (sequence_length, batch_size, EMBEDDING_DIM)
# Verifica los vectores de embeddings para los primeros 3 tokens de la primera muestra del lote
# se obtienen embeddings[i,0,:] donde i se refiere al i-ésimo token de la primera muestra en el lote (b=0)
for i in range(3):
    print(f"Embeddings de tokens para {i}-ésimo token de la primera muestra: {t_embeddings[i,0,:]}")

In [ ]:
positional_encoding = PositionalEncoding(emb_size=EMBEDDING_DIM,dropout=0)

# Aplica codificación posicional a los embeddings de tokens
p_embedding = positional_encoding(t_embeddings)

print(f"Dimension de los tokens codificados posicionalmente: {p_embedding.size()}")# Esperado: (sequence_length, batch_size, EMBEDDING_DIM)
# Verifica los vectores codificados posicionalmente para los primeros 3 tokens de la primera muestra del lote
# se obtienen encoded_tokens[i,0,:] donde i se refiere al i-ésimo token de la primera muestra (b=0) en el lote
for i in range(3):
    print(f"Embedding posicional para el {i}-ésimo token de la primera muestra: {p_embedding[i,0,:]}")

In [ ]:
segment_embedding = nn.Embedding(3, EMBEDDING_DIM)
s_embedding = segment_embedding(segment_labels)
print(f"Dimension de embeddings de segmento: {s_embedding.size()}")# Esperado: (sequence_length, batch_size, EMBEDDING_DIM)
# Verifica los vectores de embeddings de segmento para los primeros 3 tokens de la primera muestra del lote
# se obtienen segment_embedded[i,0,:] donde i se refiere al i-ésimo token de la primera muestra (b=0) en el lote
for i in range(3):
    print(f"Embedding de segmento para el {i}-ésimo token de la primera muestra: {s_embedding[i,0,:]}")

In [ ]:
# crea los vectores de embeddings combinados
bert_embeddings = t_embeddings + p_embedding + s_embedding
print(f"Dimension de token + posicion + token de segmento codificado : {bert_embeddings.size()}")
# Verifica los vectores de embeddings BERT para los primeros 3 tokens de la primera muestra del lote
# se obtienen bert_embeddings[i,0,:] donde i se refiere al i-ésimo token de la primera muestra (b=0) en el lote
for i in range(3):
    print(f"BERT_Embedding para el {i}-ésimo token: {bert_embeddings[i,0,:]}")

In [ ]:
class BERT(torch.nn.Module):
    
    def __init__(self, vocab_size, d_model=768, n_layers=12, heads=12, dropout=0.1):
        """
        vocab_size: Tamaño del vocabulario.
        d_model: Tamaño de los embeddings (hidden size).
        n_layers: Número de capas del Transformer.
        heads: Número de cabeceras de atención en cada capa del Transformer.
        dropout: Tasa de dropout aplicada a los embeddings y a las capas del Transformer.
        """
        super().__init__()
        self.d_model = d_model
        self.n_layers = n_layers
        self.heads = heads

        # Capa de embeddings que combina embeddings de tokens e embeddings de segmentos
        self.bert_embedding = BERTEmbedding(vocab_size, d_model, dropout)

        # Capas del codificador Transformer
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=heads, dropout=dropout, batch_first=False
        )
        self.transformer_encoder = nn.TransformerEncoder(
            self.encoder_layer, num_layers=n_layers
        )

        # Capa lineal para NSP (next sentence prediction)
        self.nextsentenceprediction = nn.Linear(d_model, 2)

        # Capa lineal para MLM (masked language modeling)
        self.masked_language = nn.Linear(d_model, vocab_size)

    def forward(self, bert_inputs, segment_labels):
        """
        bert_inputs: Tokens de entrada.
        segment_labels: IDs de segmento para distinguir los distintos segmentos en la entrada.
        mask: Máscara de atención para evitar que el modelo atienda a tokens de padding.

        return: Predicciones para la tarea de Next Sentence Prediction y para la tarea de Masked Language Modeling.
        """
        padding_mask = (bert_inputs == PAD_IDX).transpose(0, 1)
        # Genera embeddings a partir de tokens de entrada y etiquetas de segmento
        my_bert_embedding = self.bert_embedding(bert_inputs, segment_labels)

        # Pasa embeddings por el codificador Transformer
        transformer_encoder_output = self.transformer_encoder(
            my_bert_embedding, src_key_padding_mask=padding_mask
        )

        next_sentence_prediction = self.nextsentenceprediction(
            transformer_encoder_output[0, :]
        )
        
        # Modelado de lenguaje enmascarado: predecir todos los tokens en la secuencia
        masked_language = self.masked_language(transformer_encoder_output)

        return next_sentence_prediction, masked_language


Creamos una instancia del modelo:


In [ ]:
EMBEDDING_DIM = 10

# Define parámetros
vocab_size = 147161  # Reemplaza VOCAB_SIZE con el tamaño de tu vocabulario
d_model = EMBEDDING_DIM  # Reemplaza EMBEDDING_DIM con la dimensión de tu embeddings
n_layers = 2  # Número de capas del Transformer
initial_heads = 12  # Número inicial de cabeceras de atención
initial_heads = 2
# Asegura de que el número de cabeceras sea un factor de la dimensión del embedding
heads = initial_heads - d_model % initial_heads

dropout = 0.1  # Tasa de dropout

# Crea una instancia del modelo BERT
modelo = BERT(vocab_size, d_model, n_layers, heads, dropout)

In [ ]:
padding_mask = (bert_inputs == PAD_IDX).transpose(0, 1)
padding_mask.shape

In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=heads, dropout=dropout,batch_first=False)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
# Pasa los embeddings a través del codificador Transformer
transformer_encoder_output = transformer_encoder(bert_embeddings,src_key_padding_mask=padding_mask)
transformer_encoder_output.shape

In [ ]:
nextsentenceprediction = nn.Linear(d_model, 2)
nsp = nextsentenceprediction(transformer_encoder_output[ 0,:])
#logits para la tarea NSP
print(f"Forma de la salida NSP: {nsp.shape}")  # Forma esperada: (batch_size, 2)

In [ ]:
masked_language = nn.Linear(d_model, vocab_size)
# Modelado de lenguaje enmascarado: Predecir todos los tokens en la secuencia
mlm = masked_language(transformer_encoder_output)
#logits para tareas MLM
print(f"Forma de salida MLM: {mlm.shape}")  # Forma esperada: (seq_length, batch_size, vocab_size)

### **Evaluación**

Después de crear el modelo BERT, el siguiente paso es entrenarlo y evaluar su desempeño. Para facilitar esto, se define una función `evaluate` con los siguientes pasos:

1. **Función de pérdida**: Se define la función `CrossEntropyLoss` para calcular la pérdida entre los valores predichos y reales.
2. **Argumentos de la función**: La función recibe argumentos incluyendo el dataloader, el modelo, la función de pérdida y el dispositivo.
3. **Modo de evaluación**: El modelo BERT se pone en modo evaluación usando `modelo.eval()`, deshabilitando dropout y comportamientos específicos de entrenamiento. Se inicializan variables para rastrear la pérdida total, la pérdida total de siguiente oración, la pérdida total de máscara y el número total de lotes.
4. **Bucle de evaluación**: La función itera sobre los lotes proporcionados por el dataloader.
5. **Paso hacia adelante**: Se realiza un forward pass con el modelo BERT para obtener predicciones para las tareas de siguiente oración y modelado de lenguaje enmascarado.
6. **Cálculo de la pérdida**: Se calculan las pérdidas para las tareas de siguiente oración y lenguaje enmascarado, y luego se suman para obtener la pérdida total del lote.
7. **Cálculo de la pérdida promedio**: Se calcula la pérdida promedio, la pérdida promedio de siguiente oración y la pérdida promedio de máscara dividiendo las pérdidas totales por el número total de lotes.

La función `evaluate` se usa no solo para evaluar el desempeño del modelo BERT, sino también durante la fase de entrenamiento para valorar el progreso del modelo.


In [ ]:
PAD_IDX=0
## La función de pérdida debe ignorar los tokens PAD y calcular la pérdida solo para los tokens enmascarados
loss_fn_mlm = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
loss_fn_nsp = nn.CrossEntropyLoss()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo.to(device)
device

In [ ]:
def evaluate(dataloader=test_dataloader, modelo=modelo, loss_fn_mlm=loss_fn_mlm, loss_fn_nsp=loss_fn_nsp, device=device):
    modelo.eval()  # Desactiva dropout y otros comportamientos específicos de entrenamiento

    total_loss = 0
    total_next_sentence_loss = 0
    total_mask_loss = 0
    total_batches = 0
    with torch.no_grad():  # Desactiva gradientes para la validación, ahorra memoria y cómputo
        for batch in dataloader:
            bert_inputs, bert_labels, segment_labels, is_nexts = [b.to(device) for b in batch]

            # Paso hacia adelante
            next_sentence_prediction, masked_language = modelo(bert_inputs, segment_labels)

            # Calcula la pérdida para la predicción de la siguiente oración
            # Asegura de que is_nexts tenga la forma correcta para CrossEntropyLoss
            next_loss = loss_fn_nsp(next_sentence_prediction, is_nexts.view(-1))

            # Calcula la pérdida para predecir los tokens enmascarados
            # Aplana tanto las predicciones masked_language como bert_labels para cumplir con los requisitos de entrada de CrossEntropyLoss
            mask_loss = loss_fn_mlm(masked_language.view(-1, masked_language.size(-1)), bert_labels.view(-1))

            # Suma las dos pérdidas
            loss = next_loss + mask_loss
            if torch.isnan(loss):
                continue
            else:
                total_loss += loss.item()
                total_next_sentence_loss += next_loss.item()
                total_mask_loss += mask_loss.item()
                total_batches += 1

    avg_loss = total_loss / (total_batches + 1)
    avg_next_sentence_loss = total_next_sentence_loss / (total_batches + 1)
    avg_mask_loss = total_mask_loss / (total_batches + 1)

    print(f"Pérdida promedio: {avg_loss:.4f}, Pérdida promedio de la siguiente oración: {avg_next_sentence_loss:.4f}, Pérdida promedio de la máscara: {avg_mask_loss:.4f}")

### **Entrenamiento**

El proceso de entrenamiento del modelo BERT implica los siguientes pasos:

1. **Definición del optimizador**: Antes de comenzar el entrenamiento, se define un optimizador para entrenar el modelo BERT. En este caso, se utiliza el optimizador Adam.
2. **Bucle de entrenamiento**: Dentro de cada época, se iteran los datos de entrenamiento en lotes.
3. **Paso hacia adelante**: Para cada lote, se realiza un forward pass donde el modelo BERT predice la tarea de siguiente oración y la de lenguaje enmascarado.
4. **Cálculo de la pérdida y actualización de parámetros**: Se calcula la pérdida en función de los valores predichos y reales. Luego, los parámetros del modelo se actualizan mediante backpropagation y recorte de gradientes.
5. **Evaluación por época**: Después de cada época, se imprime la pérdida de entrenamiento promedio. Se evalúa el desempeño del modelo en el conjunto de prueba. Además, el modelo se guarda después de cada época.

Estos pasos se repiten durante múltiples épocas para entrenar el modelo BERT y monitorear su progreso a lo largo del tiempo.


**NOTA: El DataLoader actual es bastante grande y tomará varias horas entrenar el modelo con un conjunto de datos tan grande. Por lo tanto, a continuación está el conjunto de datos muestreado aleatoriamente (que es relativamente pequeño y aún tarda de 1 a 2 horas en ejecutarse) de IMDB para acelerar el proceso. Si deseas entrenar el modelo con el conjunto de datos completo, omite la siguiente celda y ejecuta directamente la celda de entrenamiento.**


In [ ]:
BATCH_SIZE = 3

train_dataset_path = './bert_dataset/bert_dataset/bert_train_data_sampled.csv'
test_dataset_path = './bert_dataset/bert_dataset/bert_test_data_sampled.csv'

train_dataset = BERTCSVDataset(train_dataset_path)
test_dataset = BERTCSVDataset(test_dataset_path)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [ ]:
# Define el optimizador
optimizer = Adam(modelo.parameters(), lr=1e-4, weight_decay=0.01, betas=(0.9, 0.999))

# Configuración del bucle de entrenamiento
num_epochs = 1
total_steps = num_epochs * len(train_dataloader)

# Define el número de pasos de calentamiento (warmup), p. ej., el 10 % del total
warmup_steps = int(total_steps * 0.1)

# Crea el programador de tasa de aprendizaje (learning rate scheduler)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

# Listas para almacenar las pérdidas para graficar
train_losses = []
eval_losses = []

for epoch in tqdm(range(num_epochs), desc="Épocas de entrenamiento"):
    modelo.train()
    total_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader, desc=f"Epoca {epoch + 1}")):
        bert_inputs, bert_labels, segment_labels, is_nexts = [b.to(device) for b in batch]

        optimizer.zero_grad()
        next_sentence_prediction, masked_language = modelo(bert_inputs, segment_labels)

        next_loss = loss_fn_nsp(next_sentence_prediction, is_nexts)
        mask_loss = loss_fn_mlm(masked_language.view(-1, masked_language.size(-1)), bert_labels.view(-1))

        loss = next_loss + mask_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(modelo.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()  # Actualiza la tasa de aprendizaje

        total_loss += loss.item()

        if torch.isnan(loss):
            continue
        else:
            total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader) + 1
    train_losses.append(avg_train_loss)
    print(f"Epoca {epoch+1} - Pérdida de entrenamiento promedio: {avg_train_loss:.4f}")

    # Evaluación tras cada época
    eval_loss = evaluate(test_dataloader, modelo, loss_fn_nsp, loss_fn_mlm, device)
    eval_losses.append(eval_loss)

**A continuación se muestra un gráfico de pérdida versus época; ejecuta el código anterior para más de una época para obtener un gráfico (actualmente, num_epochs está configurado en 1).**


In [ ]:
# Grafica los valores de pérdida por época
plt.plot(range(1, num_epochs + 1), train_losses, label='Pérdida de Entrenamiento')
plt.plot(range(1, num_epochs + 1), eval_losses, label='Pérdida de Evaluación')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.title('Pérdida de entrenamiento y evaluación')
plt.legend()
plt.show()

### **Inferencia**


In [ ]:
from transformers import BertTokenizer, BertForPreTraining 
# 1) Carga el tokenizer y ajusta vocab_size dinámicamente
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size  # 30522

# 2) Instancia el modelo con el vocab_size correcto
modelo = BertForPreTraining.from_pretrained('bert-base-uncased') 

# 3) Mueve a dispositivo y pásalo a modo evaluación
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo.to(device)
modelo.eval()


Para evaluar el rendimiento de un modelo BERT preentrenado en la tarea de predecir si una segunda oración sigue a la primera (Next Sentence Prediction, NSP), se define la función `predict_nsp`. La función opera de la siguiente manera:

1. **Tokenización**: Las oraciones de entrada se tokenizan usando `tokenizer.encode_plus`, que devuelve un diccionario con las entradas tokenizadas. Estas entradas se convierten en tensores y se envían al dispositivo adecuado para el procesamiento.
2. **Predicción**: Se utiliza el modelo BERT para hacer predicciones pasando los tensores de tokens y de segmentos como entrada.
3. **Manipulación de logits**: Se selecciona el primer elemento del tensor de logits y se añade una dimensión extra, obteniendo una forma `[1, 2]`.
4. **Probabilidad y predicción**: Los logits se pasan por una función softmax para obtener probabilidades, y la predicción se obtiene con `argmax`.
5. **Interpretación del resultado**: La predicción se interpreta y se devuelve como una cadena, indicando si la segunda oración sigue o no a la primera.
6. **Ejemplo de uso**: Se muestra cómo pasar dos oraciones de ejemplo a la función `predict_nsp` junto con el modelo y el tokenizador. El resultado se imprime, indicando si la segunda oración sigue a la primera según la predicción del modelo.

Al utilizar `predict_nsp`, puedes evaluar el desempeño del modelo BERT preentrenado en determinar la relación entre dos oraciones.


In [ ]:
def predict_nsp(sentence1, sentence2, modelo, tokenizer):
    tokens = tokenizer.encode_plus(sentence1, sentence2, return_tensors="pt")
    ids = tokens["input_ids"].to(device)
    types = tokens["token_type_ids"].to(device)
    with torch.no_grad():
        outputs = modelo(input_ids=ids, token_type_ids=types)
        # outputs.seq_relationship_logits es el tensor NSP [batch_size, 2]
        logits = torch.softmax(outputs.seq_relationship_logits, dim=-1)
        pred = torch.argmax(logits, dim=-1).item()
    return "La segunda oración sigue a la primera" if pred == 1 else "La segunda oración no sigue a la primera"


In [ ]:
# Ejemplo de uso
sentence1 = "The cat sat on the mat."
sentence2 = "It was a sunny day"

print(predict_nsp(sentence1, sentence2, modelo, tokenizer))

Se define una función para realizar *Masked Language Modeling* (MLM) usando un modelo BERT preentrenado. La función opera de la siguiente manera:

1. **Tokenización**: La oración de entrada se tokeniza con el tokenizador y se convierte en IDs de tokens, incluyendo los tokens especiales. La oración tokenizada se guarda en `tokens_tensor`.
2. **Etiquetas de segmentos**: Se crean etiquetas de segmento dummy llenas de ceros y se guardan en `segment_labels`.
3. **Predicción**: Se pasa `tokens_tensor` y `segment_labels` por el modelo BERT para extraer los logits de MLM como `predictions`.
4. **Índice del token enmascarado**: Se identifica la posición del token `[MASK]` usando `nonzero` y se guarda en `mask_token_index`.
5. **Índice predicho**: Se obtiene el índice predicho para el token `[MASK]` haciendo `argmax` sobre los logits de MLM en la posición correspondiente.
6. **Conversión a token**: El índice predicho se convierte de nuevo a un token con `convert_ids_to_tokens`.
7. **Oración reemplazada**: Se reemplaza el token `[MASK]` en la oración original con el token predicho, resultando en la oración final predicha.

In [ ]:
def predict_mlm(sentence, modelo, tokenizer):
    inputs = tokenizer(sentence, return_tensors="pt")
    ids = inputs.input_ids.to(device)
    mask_token_index = (ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    with torch.no_grad():
        outputs = modelo(input_ids=ids)
        # outputs.prediction_logits es el tensor MLM [batch_size, seq_len, vocab_size]
        logits = outputs.prediction_logits
        mask_logits = logits[0, mask_token_index, :]
        pred_id = torch.argmax(mask_logits, dim=-1).item()
        pred_token = tokenizer.convert_ids_to_tokens(pred_id)
    return sentence.replace(tokenizer.mask_token, pred_token, 1)


In [ ]:
# Ejemplos
print(predict_nsp("The cat sat on the mat.", "It was a sunny day", modelo, tokenizer))
print(predict_mlm("The cat sat on the [MASK].", modelo, tokenizer))

### **Ejercicios**


#### **Ejercicio 1: Predicción de la próxima oración (NSP) con BERT**

1. **Cargar el modelo preentrenado de BERT**: Importa `BertForPreTraining` y `BertTokenizer` desde `transformers` y cargar el modelo preentrenado y el tokenizador `bert-base-uncased`.
2. **Preparar la entrada de texto**: Codifica un par de oraciones utilizando el tokenizador cargado.
3. **Ejecutar NSP**: Pasa la entrada codificada a través del modelo e interpretar `seq_relationship_logits` para determinar si el modelo predice las oraciones como consecutivas.


In [ ]:
from transformers import BertForPreTraining, BertTokenizer
import torch

# Escribe tu codigo

#### **Ejercicio 2: Modelado de lenguaje enmascarado (MLM) con BERT**

1. **Inicializa el modelo y el tokenizador**:

Carga `BertForPreTraining` y `BertTokenizer` desde la librería `transformers` usando el modelo `bert-base-uncased`.

2. **Prepara la oración enmascarada**:

Escribe una oración y reemplazar una palabra por `[MASK]`. Por ejemplo, "The capital of France is [MASK]."

3. **Tokenizar y predecir**:

Tokeniza la oración enmascarada con `BertTokenizer`. Luego, introducirla en `BertForPreTraining` y usar `prediction_logits` para encontrar el token más probable que se ajuste a la máscara.

4. **Mostrar la predicción**:

Convierte el ID del token predicho a una cadena de tokens e imprimir la palabra predicha.

In [ ]:
from transformers import BertForPreTraining, BertTokenizer
import torch

# Escribe tu codigo